# Lecture 8 - 20.06.2022 
# Exercise : Pol-InSAR forest height inversion

* DLR's F-SAR acquisition over Traunstein forest (Germany)
* Path: '/projects/data/04-polinsar/'
* SLCs: 
    * Acquisition 1 : slc_15tmpsar0302_L { hh, hv, vv, vh } _t01.rat
    * Acquisition 2 : slc_15tmpsar0302_15tmpsar0303_L { hh, hv, vv, vh } _t01.rat
* Flat-earth: pha_flat_15tmpsar0302_15tmpsar0303_Lhh_t01.rat
* Vertical wavenumber (kz) : kz_2d_demc_15tmpsar0302_15tmpsar0303_t01.rat
* Lidar: Lida_r1503.rat

Objective:
- Estimate forest height using the phase extremes of the dual-pol Pol-InSAR coherence region and model inversion. 

Tips:
- work on the azimuth - range block [21500 - 4000, 21500 + 6000] - [2300, 4500] ;
- all the needed functions (including the full calculation of the coherence region) have been already implemented.

In [ ]:
# import useful libraries, functions, and modules

import sys
sys.path.append('/projects/src/')

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import filters
from ste_io import *
from tqdm import tqdm

%matplotlib widget



In [ ]:
def calculate_covariance(im1, im2, looksr, looksa) : 
    
    # ... apply definition
    corr = filters.uniform_filter(np.real(im1*np.conj(im2)), [looksa,looksr]) + 1j* \
                filters.uniform_filter(np.imag(im1*np.conj(im2)), [looksa,looksr])
    
    # ... and back to main
    return corr

In [ ]:
def make_pauli(slchh, slchv, slcvv, looksr, looksa) :
    
    # 1. Uses function calculate_covariance
    # 2. Convention (rows x columns): inputs are (az x rg), outputs are (rg x az) 
    #                                 for better plotting as #rg <#az
    
    # Calculate T11, T22 and T33
    T11 = calculate_covariance(slchh + slcvv, slchh + slcvv, looksr, looksa)
    T22 = calculate_covariance(slchh - slcvv, slchh - slcvv, looksr, looksa)
    T33 = calculate_covariance(2*slchv, 2*slchv, looksr, looksa)
    
    # make the pauli rgb (+ tranpose, clipping, normalization)
    dimaz = T11.shape[0]
    dimrg = T11.shape[1]
    rgb_pauli = np.zeros((dimrg, dimaz, 3), 'float32')
    rgb_pauli[:, :, 0] = np.transpose( np.clip(np.sqrt(np.abs(T22)), 0, 2.5*np.mean(np.sqrt(np.abs(T22)))) )    # red
    rgb_pauli[:, :, 1] = np.transpose( np.clip(np.sqrt(np.abs(T33)), 0, 2.5*np.mean(np.sqrt(np.abs(T33)))) )    # green
    rgb_pauli[:, :, 2] = np.transpose( np.clip(np.sqrt(np.abs(T11)), 0, 2.5*np.mean(np.sqrt(np.abs(T11)))) )    # blue
    rgb_pauli[:, :, 0] = rgb_pauli[:, :, 0] / np.max(rgb_pauli[:, :, 0])     # red
    rgb_pauli[:, :, 1] = rgb_pauli[:, :, 1] / np.max(rgb_pauli[:, :, 1])     # green
    rgb_pauli[:, :, 2] = rgb_pauli[:, :, 2] / np.max(rgb_pauli[:, :, 2])     # blue
    
    # ... and back to main
    return rgb_pauli

In [ ]:
def matmul_2(a11, a12, a21, a22, b11, b12, b21, b22) :
    
    # Implements the multiplication A.B of two 2 x 2 matrices A and B.
    # Inputs are the elements of matrices (row-wise). 
    # Outputs are the elements of the matrix product (row-wise).
    
    c11 = a11*b11 + a12*b21
    c12 = a11*b12 + a12*b22
    c21 = a21*b11 + a22*b21 
    c22 = a21*b12 + a22*b22
    
    # ... and back to main
    return c11, c12, c21, c22

In [ ]:
def eigenvalvect_2(a11, a12, a21, a22) :
    
    # Calculates eigenvalues / eigenvectors of a generic complex-valued 2 x 2 matrix.
    # Inputs are the matrix elements (row-wise) - NumPy arrays
    # Outputs are the eigenvalues (l1, l2), and the eigenvector (E1, E2) elements - NumPy arrays
    #                                       E1 = [e11, e21] , U2 = [e12, e22]
    
    # eigenvalues 
    l1 = 0.5 * ((a11+a22) + np.sqrt( (a11-a22)**2 + 4*a12*a21 )) 
    l2 = 0.5 * ((a11+a22) - np.sqrt( (a11-a22)**2 + 4*a12*a21 ))
    
    # eigenvector 1
    e11 = -a12
    e21 = a11 - l1
    # normalize eigenvector 1
    norm_ = np.sqrt( np.abs(e11)**2 + np.abs(e21)**2 )
    e11 = e11 / norm_
    e21 = e21 / norm_
    # eigenvector 2
    e12 = a22 - l2
    e22 = -a21
    # normalize eigenvector 2
    norm_ = np.sqrt( np.abs(e12)**2 + np.abs(e22)**2 )
    e12 = e12 / norm_
    e22 = e22 / norm_
    
    del norm_
    
    # ... and back to main
    return l1, l2, e11, e21, e12, e22

In [ ]:
def sqrt_inverse(a11, a12, a22) :

    # Given a 2 x 2 Hermitian matrix A, calculates A**(-1/2) using eigendecomposition
    # Inputs are the upper-diagonal matrix elements (row-wise) - NumPy arrays.
    # Outputs are the elements of the sqrt inverse matrix.
    # Uses function matmul_2.
    
    # Calculate eigenvalues (l1, l2)
    sigma = 0.5 * (a11 + a22)
    D = a11*a22 - a12*np.conj(a12)
    delta = np.sqrt(sigma**2 - D)
    l1 = sigma + delta
    l2 = sigma - delta
    
    # Calculate eigenvector matrix elements (eigenvectors are the columns)
    norm_ = np.sqrt( a12*np.conj(a12) + ((a11-a22)/2 - delta)**2 )
    v11 = -a12 / norm_
    v21 = ((a11-a22)/2 - delta) / norm_
    v12 = ((a11-a22)/2 - delta) / norm_
    v22 = np.conj(a12) / norm_
    
    del norm_, delta, D, sigma
    
    # calculate sqrt inverse matrix elements
    zz = np.zeros(l1.shape, 'float32')
    aux11, aux12, aux21, aux22 = matmul_2(v11, v12, v21, v22, 1/np.sqrt(l1), zz, zz, 1/np.sqrt(l2))
    del l1, l2
    i11, i12, i21, i22 = matmul_2(aux11, aux12, aux21, aux22, np.conj(v11), np.conj(v21), np.conj(v12), np.conj(v22))
    del aux11, aux12, aux21, aux22
    
    # ... and back to main
    return i11, i12, i21, i22  

In [ ]:
def optimize_coherence_and_phase(P11, P12, P21, P22):
    """
    Performs the Polar decomposition of the 2 by 2 matrix P with the given elements
    and estimates coherences with min max phase and absolute value.
    
    The polar decomposition of P is P = U * J.
    The max & min coherence is extracted from the eigenvectors of J and the
    max & min phase from the eigenvectors of U
    
    See: https://en.wikipedia.org/wiki/Polar_decomposition
    """
    
    # Calculate J^2 = P^(*T) * P
    J11, J12, J21, J22 = matmul_2(np.conj(P11), np.conj(P21), np.conj(P12), np.conj(P22), P11, P12, P21, P22)
    # Calculate eigenvalues and eigenvectors of J
    l1, l2, v11, v21, v12, v22 = eigenvalvect_2(J11, J12, J21, J22)
    del l1, l2
    
    # Calculate V^(*T) * P * V --> gamma max & min from there
    aux11, aux12, aux21, aux22 = matmul_2(np.conj(v11), np.conj(v21), np.conj(v12), np.conj(v22), P11, P12, P21, P22)
    gamma_max, gamma12, gamma21, gamma_min = matmul_2(aux11, aux12, aux21, aux22, v11, v12, v21, v22)
    del aux11, aux12, aux21, aux22, v11, v21, v12, v22
    del gamma12, gamma21
    
    # Compute U = P * J^(-1)
    Ji11, Ji1, Ji21, Ji22 = sqrt_inverse(J11, J12, J22)
    del J11, J12, J21, J22
    U11, U12, U21, U22 = matmul_2(P11, P12, P21, P22, Ji11, Ji1, Ji21, Ji22)
    del Ji11, Ji1, Ji21, Ji22
    
    # Calculate eigenvalues and eigenvectors of U
    l1, l2, w11, w21, w12, w22 = eigenvalvect_2(U11, U12, U21, U22)
    del l1, l2, U11, U12, U21, U22
    
    # Calculate W^(*T) * P * W --> gamma phamax & phamin from there
    aux11, aux12, aux21, aux22 = matmul_2(np.conj(w11), np.conj(w21), np.conj(w12), np.conj(w22), P11, P12, P21, P22)
    gamma_phamax, gamma12, gamma21, gamma_phamin = matmul_2(aux11, aux12, aux21, aux22, w11, w12, w21, w22)
    del aux11, aux12, aux21, aux22, w11, w21, w12, w22
    del gamma12, gamma21
    
    # Return optimum coherences
    return gamma_phamax, gamma_phamin, gamma_max, gamma_min


In [ ]:
def plot_coherence_region_P(P11, P12, P21, P22, npoints=128, axes=None, color='r', **kwargs):
    # Plots the coherence region of the pre-whitened matrix P
    M = np.asarray([[P11, P12], [P21, P22]])
    theta = np.linspace(0, np.pi, npoints)
    Wi,Vi = np.linalg.eigh(0.5*(M[None,...]*np.exp(1j*theta)[...,None,None] + np.conj(M.T)[None,...]*np.exp(-1j*theta)[...,None,None]))

    if axes is None:
        ax = plt.subplot(111, projection='polar')
    else:
        ax = axes
    indices = np.argsort(Wi, axis=-1)
    Z = np.einsum('...i,...ij,...j->...', np.conj(Vi[range(npoints),:,indices[:,-1]]), M, Vi[range(npoints),:,indices[:,-1]])
    Z2 = np.einsum('...i,...ij,...j->...', np.conj(Vi[range(npoints),:,indices[:,0]]), M, Vi[range(npoints),:,indices[:,0]])
    ax.plot(np.angle(Z), np.abs(Z), color, **kwargs)
    ax.plot(np.angle(Z2), np.abs(Z2), color, **kwargs)
    return ax

**Input parameters**

In [ ]:
# --- Inputs

# path 2 images
path = '/projects/data/04-polinsar/'

# Input pixel spacing, in meters
spacrg = 0.59941552
spacaz = 0.19507939

# Output range resolution, in meters
resrg = 5.
resaz = 5.

# Image block for processing
minrg = 2300
maxrg = 4500
minaz = 21500 - 4000
maxaz = 21500 + 6000


In [ ]:
# --- Calculate number of looks

looksr = int( resrg / spacrg )
if looksr % 2 == 0 : looksr = looksr +1
looksa = int( resaz / spacaz )
if looksa % 2 == 0 : looksa = looksa +1


**Step 1: Open images, and visualize a Pauli**

In [ ]:
# --- Open images

# all channels for the first acquisition to make the the Pauli
slchh1 = rrat(path + 'slc_15tmpsar0302_Lhh_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])
slchv1 = rrat(path + 'slc_15tmpsar0302_Lhv_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])
slcvv1 = rrat(path + 'slc_15tmpsar0302_Lvv_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])

# only relevant channels for the second acquisition
slchh2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lhh_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])
slchv2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lhv_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])
slcvv2 = rrat(path + 'slc_coreg_15tmpsar0302_15tmpsar0303_Lvv_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])


**Step 2 : Compensate flat-earth**

In [ ]:
# --- Open flat-earth phase
fe = rrat(path + 'pha_flat_15tmpsar0302_15tmpsar0303_Lhh_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])

# --- Compensate
slchh2 = slchh2 * np.exp(1j * fe)
slchv2 = slchv2 * np.exp(1j * fe)
slcvv2 = slcvv2 * np.exp(1j * fe)

# free memory
del fe


**Step 3 : Calculate the elements of T1, T2, and Omega matrices**

In [ ]:
# --- create channels
im1_1 = slchh1 - slcvv1
im2_1 = 2*slchv1
im1_2 = slchh2 - slcvv2
im2_2 = 2*slchv2 

# free memory
del slchh1, slchv1, slcvv1, slchh2, slchv2, slcvv2

# --- T1 
T1_11 = calculate_covariance(im1_1, im1_1, looksr, looksa)
T1_12 = calculate_covariance(im1_1, im2_1, looksr, looksa)
T1_22 = calculate_covariance(im2_1, im2_1, looksr, looksa)

# --- T2 
T2_11 = calculate_covariance(im1_2, im1_2, looksr, looksa)
T2_12 = calculate_covariance(im1_2, im2_2, looksr, looksa)
T2_22 = calculate_covariance(im2_2, im2_2, looksr, looksa)

# --- Omega
Om_11 = calculate_covariance(im1_1, im1_2, looksr, looksa)
Om_12 = calculate_covariance(im1_1, im2_2, looksr, looksa)
Om_21 = calculate_covariance(im2_1, im1_2, looksr, looksa)
Om_22 = calculate_covariance(im2_1, im2_2, looksr, looksa)

# free memory
del im1_1, im2_1, im1_2, im2_2

# --- polarization 1 and polarization 2 InSAR coherences
gamma1 = Om_11 / np.sqrt(T1_11) / np.sqrt(T2_11)
gamma2 = Om_22 / np.sqrt(T1_22) / np.sqrt(T2_22)


**Step 4 : Get optimized coherences and Maximum/minimum phase**

In [ ]:
# --- calculate T

T11 = 0.5 * (T1_11 + T2_11)
T12 = 0.5 * (T1_12 + T2_12)
T22 = 0.5 * (T1_22 + T2_22)

# free memory
del T1_11, T1_12, T1_22, T2_11, T2_12, T2_22

# --- Pre-whitening
# calculate the elements of T**(-1/2)
iT11, iT12, iT21, iT22 = sqrt_inverse(T11, T12, T22)

# free memory
del T11, T12, T22

# Calculate Pi matrix
# whiten ... T^(-1/2) * Om * T^(-1/2) ==> Matrix P
aux11, aux12, aux21, aux22 = matmul_2(iT11, iT12, iT21, iT22, Om_11, Om_12, Om_21, Om_22)
del Om_11, Om_12, Om_21, Om_22
P11, P12, P21, P22 = matmul_2(aux11, aux12, aux21, aux22, iT11, iT12, iT21, iT22)
del aux11, aux12, aux21, aux22
del iT11, iT12, iT21, iT22


In [ ]:
### Use maximum and minimum phase for line fitting
gamma_phamax, gamma_phamin, gamma_max, gamma_min = optimize_coherence_and_phase(P11, P12, P21, P22)

**Step 5 : Find the volume-only coherence**

In [ ]:
# --- load kz
kz = rrat(path + 'kz_2d_demc_15tmpsar0302_15tmpsar0303_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])


In [ ]:
# --- decide gamma_v


**Step 6 : Calculate ground phase**

In [ ]:
# --- calculate ground phase

v = gamma_phamin - gamma_v
x = -np.real( np.conj(v)*gamma_v ) / np.abs(v)**2 + \
            np.sqrt( np.real(np.conj(v)*gamma_v)**2 - (np.abs(gamma_v)**2 - 1)* np.abs(v)**2 ) / np.abs(v)**2
cohg = gamma_v + v*x
phi0  = np.angle(cohg)

# ... display



**Step 7 : Visualize coherence region in 1 pixel**

In [ ]:
azp = 6000
rgp = 500

P11_px = P11[azp, rgp]
P12_px = P12[azp, rgp]
P21_px = P21[azp, rgp]
P22_px = P22[azp, rgp]

# Plot coherence region and line before compensating phig

# Plot coherence region and line after compensating phig


**Step 8 : Compensate ground phase** 

**Step 9 : Plot look-up table for the chosen pixel**

In [ ]:
# --- load incidence angle

incang = rrat(path + 'incidence_15tmpsar0302_L_t01.rat', block = [minaz, maxaz+1, minrg, maxrg+1])

# --- Define interval of variation of height and extinction

H = np.linspace(0, 50, 51)  # height
#s = np.linspace(0, 2, 31)   # extinction - linear sampling
#s = np.logspace(-6, 0, 31)   # extinction - log sampling
s = np.tan(np.linspace(0.000001, .99, 31) * np.pi/2) / 30 

# make meshgrid
Hmat, smat = np.meshgrid(H, s)

# --- calculate LUT = look-up table = all the coherences for every combination of height and extinction
p1 = 2*smat/np.cos(incang[azp, rgp])
p2 = 2*smat/np.cos(incang[azp, rgp]) + 1j*kz[azp, rgp]
lut = ( p1 * (np.exp(p2*Hmat)-1) ) / ( p2 * (np.exp(p1*Hmat)-1))
lut[Hmat == 0] = 1
HH = Hmat[smat < 1e-6]
lut[smat < 1e-6] = np.sinc(kz[azp, rgp]*HH/2/np.pi) * np.exp(1j*kz[azp, rgp]*HH/2 )


# Make a plot of the LUT & gamma_v0


**Step 10 : Forest height inversion & validation**

In [ ]:
# Estimate forest height

In [ ]:
# Load Lidar data
Hl = rrat(path + 'Lida_r1503.rat', block = [minaz, maxaz+1, minrg, maxrg+1])

In [ ]:
# --- display estimates


In [ ]:
# --- validation
